In [1]:
from lightning.pytorch.loggers import CSVLogger, WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from loaders.Sdataloader import Stanford_Dataset
from model.GNN_inf_seg import Lightning_GNN
from sklearn.model_selection import train_test_split
import torch_geometric as tg
import numpy as np
import lightning as pl
import datetime
import yaml
import os
import wandb
import torch
#wandb.login(key='446bb0e42e6ee0d7b7a2224d3b524a036009d8ad')
# wandb.login()

/home/lars/anaconda3/envs/PCloud/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
# Load array with params from config.yml
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)

# Data setup
dataset_test = Stanford_Dataset(root='Stanford3dDataset',
                                 split='test')

# dataset_val = Stanford_Dataset(root=config['root'])

test_loader = tg.loader.DataLoader(dataset_test,
                                    batch_size=config['batch_size'],
                                    num_workers=2,
                                    shuffle=False)

In [3]:
state_dict = torch.load('/home/lars/output_seg/out_2/epoch=1510-train_loss=0.29.ckpt')['state_dict']

In [7]:
state_dict.keys()

odict_keys(['model.linear.weight', 'model.linear.bias', 'model.pconv_in.linear_up.weight', 'model.pconv_in.linear_up.bias', 'model.pconv_in.attn.lins.0.weight', 'model.pconv_in.attn.lins.0.bias', 'model.pconv_in.attn.lins.1.weight', 'model.pconv_in.attn.lins.1.bias', 'model.pconv_in.attn.norms.0.module.weight', 'model.pconv_in.attn.norms.0.module.bias', 'model.pconv_in.attn.norms.0.module.running_mean', 'model.pconv_in.attn.norms.0.module.running_var', 'model.pconv_in.attn.norms.0.module.num_batches_tracked', 'model.pconv_in.pos.lins.0.weight', 'model.pconv_in.pos.lins.0.bias', 'model.pconv_in.conv.pos_nn.lins.0.weight', 'model.pconv_in.conv.pos_nn.lins.0.bias', 'model.pconv_in.conv.attn_nn.lins.0.weight', 'model.pconv_in.conv.attn_nn.lins.0.bias', 'model.pconv_in.conv.attn_nn.lins.1.weight', 'model.pconv_in.conv.attn_nn.lins.1.bias', 'model.pconv_in.conv.attn_nn.norms.0.module.weight', 'model.pconv_in.conv.attn_nn.norms.0.module.bias', 'model.pconv_in.conv.attn_nn.norms.0.module.runni

In [4]:
# Model setup
GNN_model = Lightning_GNN(config=config)

GNN_model.load_state_dict(torch.load('/home/lars/output_seg/out_2/epoch=1470-train_loss=0.30.ckpt')['state_dict'])

GNN_model.to('cpu')

Lightning_GNN(
  (model): TransformerGNN(
    (linear): Linear(in_features=3, out_features=32, bias=True)
    (pconv_in): PointTrans_Layer(
      (linear_up): Linear(in_features=32, out_features=32, bias=True)
      (attn): MLP(32, 32, 32)
      (pos): MLP(3, 32)
      (conv): PointTransformerConv(32, 32)
    )
    (enc1): Enc_block(
      (downlayer): PointTrans_Layer_down(
        (linear): Linear(in_features=32, out_features=64, bias=True)
        (down): Sequential(
          (0): Linear(in_features=32, out_features=64, bias=True)
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (pconv): PointTrans_Layer(
        (linear_up): Linear(in_features=64, out_features=64, bias=True)
        (attn): MLP(64, 64, 64)
        (pos): MLP(3, 64)
        (conv): PointTransformerConv(64, 64)
      )
    )
    (enc2): Enc_block(
      (downlayer): PointTrans_Layer_down(
        (linear): Linear(in_features=

In [5]:
# Test
trainer = pl.Trainer(max_epochs=1,
                     accelerator='cpu',
                     log_every_n_steps=1)

trainer.test(GNN_model,
            dataloaders=test_loader) 

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/lars/anaconda3/envs/PCloud/lib/python3.12/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/home/lars/anaconda3/envs/PCloud/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Testing DataLoader 0: 100%|██████████| 34/34 [00:36<00:00,  0.93it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7687121033668518
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.7687121033668518}]